<a href="https://colab.research.google.com/github/AlAntonov/poultry/blob/main/get_number_of_clusters_dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
# download good sample image
!gdown --id 17uDkD5dt5JIr0z7nER0TJ3--_I-7FwQ8

In [ ]:
# download our video
!gdown --id 1r1E2NZ9r3AodYm-7PYR4mVUthRfb7fNa

In [ ]:
# import the necessary packages
from skimage import morphology
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage
import numpy as np
import cv2
import os
import sys
from os import remove
import pandas as pd
import ntpath
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython import display
from scipy import ndimage as ndi

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_number_of_clusters(image, size_threshold, number_of_clusters_list):
  height, width = image.shape[:2]
  # introduce a scale factor in order to operate with smaller resolutions
  scale_factor = height * width / (1280 * 720)
  
  # the filtering stage of meanshift segmentation
  # output is the filtered "posterized" image with color
  # gradients and fine-grain texture flattened
  aperture = 15 # aperture of median blur filter (depends on conditions)
  image_mean = cv2.medianBlur(image, aperture)
  # convert the mean shift image to grayscale, then apply
  # Otsu's thresholding
  image_gray = cv2.cvtColor(image_mean, cv2.COLOR_BGR2GRAY)
  threshold = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

  # compute the exact Euclidean distance from every binary
  # pixel to the nearest zero pixel, then find peaks in this
  # distance map
  distance_map = ndimage.distance_transform_edt(threshold)
  # here min_distance is also empirical parameter
  local_max = peak_local_max(distance_map, indices=False, min_distance=int(20 * scale_factor), labels=threshold)

  distance = ndi.distance_transform_edt(image)

  # perform a connected component analysis on the local peaks,
  # using 8-connectivity, then appy the Watershed algorithm
  markers = ndimage.label(local_max, structure=np.ones((3, 3)))[0]
  labels = watershed(-distance_map, markers, mask=threshold)

  # remove noise labels with size < 400 (empirical parameter also)
  size_threshold = size_threshold
  labels = morphology.remove_small_objects(labels, size_threshold * scale_factor)
  number_of_clusters = len(np.unique(labels)) - 1;
  number_of_clusters_list.append(number_of_clusters)
  # print(number_of_clusters)
  
  fig, axes = plt.subplots(ncols=3, figsize=(27, 9))#, sharex=True, sharey=True)
  ax = axes.ravel()

  ax[0].imshow(image, cmap=plt.cm.gray)
  ax[0].set_title('Overlapping objects')
  # ax[1].imshow(-distance, cmap=plt.cm.gray)
  # ax[1].set_title('Distances')
  ax[1].imshow(labels, cmap=plt.cm.nipy_spectral)
  ax[1].set_title('Separated objects')
  ax[2].plot(number_of_clusters_list, 'b')
  ax[2].set_title('Time frame')

  # for a in ax:
  #  a.set_axis_off()

  fig.tight_layout()
  display.display(plt.gcf())
  display.clear_output(wait=True)
  
  return number_of_clusters_list

In [ ]:
def get_video_frame_num(video, frame_num):
    cap = cv2.VideoCapture(video)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num-1)
    res, frame = cap.read()

    return frame

In [ ]:
def crop_image(img):
  # return img[int(img.shape[0]/6):int(3*img.shape[0]/6), int(img.shape[1]/3):int(2*img.shape[1]/3)]
  # return img[int(img.shape[0]/3):int(2*img.shape[0]/3), int(img.shape[1]/3):int(2*img.shape[1]/3)]
  return img[int(img.shape[0]/6):int(5*img.shape[0]/6), int(2*img.shape[1]/4):int(3*img.shape[1]/4)]

In [ ]:
# show dynamics
number_of_clusters_list = []
start = 4555
step = 5
frame_quantity = 60
size_threshold = 3200

for frame_num in range(start, start+frame_quantity*step, step):
  image = get_video_frame_num('/content/20210316-100031.mp4', frame_num)
  image = crop_image(image)
  number_of_clusters_list = get_number_of_clusters(image, size_threshold, number_of_clusters_list)